In [20]:
# Setting Up Dependencies
import requests
import pandas as pd
import matplotlib.pyplot as plt
import time
import gmaps
from config import api_key
import numpy as np

gkey = api_key
gmaps.configure(api_key=gkey)

In [24]:
number_of_breweries = 1000
brewery_id = []

brewery_id.append(np.random.rand(0, 6000))

brewery_id



[array([], shape=(0, 6000), dtype=float64)]

In [15]:
# Retrieving Raw Brewery Data

raw_brewery_data = []
start = time.time()
for x in range(number_of_breweries):
    try:
        url = "https://api.openbrewerydb.org/breweries/" + brewery_id
        response = requests.get(url).json()
        raw_brewery_data.append(response)
        print(f"({x+1}/{number_of_breweries}) Retrieved Brewery ID {x+1}...")

    except:
        print(f"({x+1}/{number_of_breweries}) Timeout for Brewery ID {x+1}... skipping")
        
stop = time.time()
time_elapsed = round((stop - start)/60,2)
print(f"\nRetrieved {number_of_breweries} breweries... Time elapsed: {time_elapsed} minutes.")

(1/1000) Retrieved Brewery ID 1...
(2/1000) Retrieved Brewery ID 2...
(3/1000) Retrieved Brewery ID 3...
(4/1000) Retrieved Brewery ID 4...
(5/1000) Retrieved Brewery ID 5...
(6/1000) Retrieved Brewery ID 6...
(7/1000) Retrieved Brewery ID 7...
(8/1000) Retrieved Brewery ID 8...
(9/1000) Retrieved Brewery ID 9...
(10/1000) Retrieved Brewery ID 10...
(11/1000) Retrieved Brewery ID 11...
(12/1000) Retrieved Brewery ID 12...
(13/1000) Retrieved Brewery ID 13...
(14/1000) Retrieved Brewery ID 14...
(15/1000) Retrieved Brewery ID 15...
(16/1000) Retrieved Brewery ID 16...
(17/1000) Retrieved Brewery ID 17...
(18/1000) Retrieved Brewery ID 18...
(19/1000) Retrieved Brewery ID 19...
(20/1000) Retrieved Brewery ID 20...
(21/1000) Retrieved Brewery ID 21...
(22/1000) Retrieved Brewery ID 22...
(23/1000) Retrieved Brewery ID 23...
(24/1000) Retrieved Brewery ID 24...
(25/1000) Retrieved Brewery ID 25...
(26/1000) Retrieved Brewery ID 26...
(27/1000) Retrieved Brewery ID 27...
(28/1000) Retrieved

(217/1000) Retrieved Brewery ID 217...
(218/1000) Retrieved Brewery ID 218...
(219/1000) Retrieved Brewery ID 219...
(220/1000) Retrieved Brewery ID 220...
(221/1000) Retrieved Brewery ID 221...
(222/1000) Retrieved Brewery ID 222...
(223/1000) Retrieved Brewery ID 223...
(224/1000) Retrieved Brewery ID 224...
(225/1000) Retrieved Brewery ID 225...
(226/1000) Retrieved Brewery ID 226...
(227/1000) Retrieved Brewery ID 227...
(228/1000) Retrieved Brewery ID 228...
(229/1000) Retrieved Brewery ID 229...
(230/1000) Retrieved Brewery ID 230...
(231/1000) Retrieved Brewery ID 231...
(232/1000) Retrieved Brewery ID 232...
(233/1000) Retrieved Brewery ID 233...
(234/1000) Retrieved Brewery ID 234...
(235/1000) Retrieved Brewery ID 235...
(236/1000) Retrieved Brewery ID 236...
(237/1000) Retrieved Brewery ID 237...
(238/1000) Retrieved Brewery ID 238...
(239/1000) Retrieved Brewery ID 239...
(240/1000) Retrieved Brewery ID 240...
(241/1000) Retrieved Brewery ID 241...
(242/1000) Retrieved Brew

(428/1000) Retrieved Brewery ID 428...
(429/1000) Retrieved Brewery ID 429...
(430/1000) Retrieved Brewery ID 430...
(431/1000) Retrieved Brewery ID 431...
(432/1000) Retrieved Brewery ID 432...
(433/1000) Retrieved Brewery ID 433...
(434/1000) Retrieved Brewery ID 434...
(435/1000) Retrieved Brewery ID 435...
(436/1000) Retrieved Brewery ID 436...
(437/1000) Retrieved Brewery ID 437...
(438/1000) Retrieved Brewery ID 438...
(439/1000) Retrieved Brewery ID 439...
(440/1000) Retrieved Brewery ID 440...
(441/1000) Retrieved Brewery ID 441...
(442/1000) Retrieved Brewery ID 442...
(443/1000) Retrieved Brewery ID 443...
(444/1000) Retrieved Brewery ID 444...
(445/1000) Retrieved Brewery ID 445...
(446/1000) Retrieved Brewery ID 446...
(447/1000) Retrieved Brewery ID 447...
(448/1000) Retrieved Brewery ID 448...
(449/1000) Retrieved Brewery ID 449...
(450/1000) Retrieved Brewery ID 450...
(451/1000) Retrieved Brewery ID 451...
(452/1000) Retrieved Brewery ID 452...
(453/1000) Retrieved Brew

(639/1000) Retrieved Brewery ID 639...
(640/1000) Retrieved Brewery ID 640...
(641/1000) Retrieved Brewery ID 641...
(642/1000) Retrieved Brewery ID 642...
(643/1000) Retrieved Brewery ID 643...
(644/1000) Retrieved Brewery ID 644...
(645/1000) Retrieved Brewery ID 645...
(646/1000) Retrieved Brewery ID 646...
(647/1000) Retrieved Brewery ID 647...
(648/1000) Retrieved Brewery ID 648...
(649/1000) Retrieved Brewery ID 649...
(650/1000) Retrieved Brewery ID 650...
(651/1000) Retrieved Brewery ID 651...
(652/1000) Retrieved Brewery ID 652...
(653/1000) Retrieved Brewery ID 653...
(654/1000) Retrieved Brewery ID 654...
(655/1000) Retrieved Brewery ID 655...
(656/1000) Retrieved Brewery ID 656...
(657/1000) Retrieved Brewery ID 657...
(658/1000) Retrieved Brewery ID 658...
(659/1000) Retrieved Brewery ID 659...
(660/1000) Retrieved Brewery ID 660...
(661/1000) Retrieved Brewery ID 661...
(662/1000) Retrieved Brewery ID 662...
(663/1000) Retrieved Brewery ID 663...
(664/1000) Retrieved Brew

(850/1000) Retrieved Brewery ID 850...
(851/1000) Retrieved Brewery ID 851...
(852/1000) Retrieved Brewery ID 852...
(853/1000) Retrieved Brewery ID 853...
(854/1000) Retrieved Brewery ID 854...
(855/1000) Retrieved Brewery ID 855...
(856/1000) Retrieved Brewery ID 856...
(857/1000) Retrieved Brewery ID 857...
(858/1000) Retrieved Brewery ID 858...
(859/1000) Retrieved Brewery ID 859...
(860/1000) Retrieved Brewery ID 860...
(861/1000) Retrieved Brewery ID 861...
(862/1000) Retrieved Brewery ID 862...
(863/1000) Retrieved Brewery ID 863...
(864/1000) Retrieved Brewery ID 864...
(865/1000) Retrieved Brewery ID 865...
(866/1000) Retrieved Brewery ID 866...
(867/1000) Retrieved Brewery ID 867...
(868/1000) Retrieved Brewery ID 868...
(869/1000) Retrieved Brewery ID 869...
(870/1000) Retrieved Brewery ID 870...
(871/1000) Retrieved Brewery ID 871...
(872/1000) Retrieved Brewery ID 872...
(873/1000) Retrieved Brewery ID 873...
(874/1000) Retrieved Brewery ID 874...
(875/1000) Retrieved Brew

In [16]:
# Creating DataFrame
brewery_df = pd.DataFrame(raw_brewery_data)
brewery_df

,brewery_type,city,country,id,latitude,longitude,name,phone,postal_code,state,street,updated_at,website_url
0,planning,Spanish Fort,United States,1,30.6749127,-87.9152724,5 Rivers Brewing LLC,2516897483,36527-3161,Alabama,,2018-08-23T23:19:56.771Z,http://5riversbrewing.com
1,micro,Birmingham,United States,2,33.524521,-86.774322,Avondale Brewing Co,2057775456,35222-1932,Alabama,201 41st St S,2018-08-23T23:19:57.825Z,http://www.avondalebrewing.com
2,micro,Gadsden,United States,3,34.016888,-86.005006,Back Forty Beer Co,2564674912,35901-3361,Alabama,200 N 6th St,2018-08-23T23:19:58.654Z,http://www.backfortybeer.com
3,micro,Tuscaloosa,United States,4,33.1984907123707,-87.5621551272424,Band of Brothers Brewing Company,2052665137,35401-4653,Alabama,1605 23rd Ave,2018-08-23T23:19:59.462Z,http://www.bandofbrosbrewing.com
4,brewpub,Huntsville,United States,5,34.7327236530612,-86.5858393061225,Below the Radar Brewing Co,2564696617,35801-4837,Alabama,220 Holmes Ave NE,2018-08-23T23:20:00.300Z,http://www.btrbrew.com
5,micro,Gulf Shores,United States,6,30.278051,-87.683039,Big Beach Brewing Company,2519482337,36542-3104,Alabama,300 E 24th Ave,2018-08-23T23:20:01.170Z,http://www.bigbeachbrewing.com
6,planning,Birmingham,United States,7,33.5206824,-86.8024326,Birmingham District Brewing,6019380486,35242-5651,Alabama,,2018-08-23T23:20:01.613Z,
7,micro,Tuscaloosa,United States,8,33.2106175454545,-87.565915,Black Warrior Brewing Co.,,35401-1542,Alabama,2216 University Blvd,2018-08-23T23:20:02.508Z,http://www.blackwarriorbrewing.com
8,micro,Madison,United States,9,None,None,Blue Pants Brewery,2566796330,35758-8675,Alabama,"500 Lanier Rd, Bld 1 - Ste A",2018-08-11T21:35:43.498Z,http://www.bluepantsbrew.com
9,micro,Florence,United States,10,34.8041295,-87.6597052887684,Bluewater Brewing Co,2562751221,35630-4810,Alabama,318 S Royal Ave,2018-08-23T23:20:04.204Z,http://www.facebook.com/pg/BluewaterBrewingCom...


In [17]:
# Drop rows with "None" Lats/Lngs
brewery_df = brewery_df[brewery_df.latitude.str.contains("None") == False]
print(f"Dropped {number_of_breweries - len(brewery_df)} breweries from DataFrame which did not have coordinates.")

Dropped 311 breweries from DataFrame which did not have coordinates.


In [18]:
brewery_loc = brewery_df[['latitude','longitude']].astype(float)
brewery_loc.head()

,latitude,longitude
0,30.674913,-87.915272
1,33.524521,-86.774322
2,34.016888,-86.005006
3,33.198491,-87.562155
4,34.732724,-86.585839


In [19]:
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(brewery_loc, dissipating=False, max_intensity=10, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))